In [1]:
import requests
from bs4 import BeautifulSoup
import time
import sys
import pandas as pd 
from pandas import DataFrame
from requests.utils import parse_header_links
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from datetime import datetime, timedelta
import csv
from airflow import DAG
from airflow.operators.python import PythonOperator

from datetime import date

C:\Users\ADMIN\anaconda3\lib\site-packages\airflow\models\base.py:49 MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)

In [9]:
def extract():
    browser = webdriver.Chrome() # Mở 1 trình duyệt Chrome
    url = "https://careerbuilder.vn/viec-lam-tuong-tu/HCM-Fresher-Data-Scientist-tai-ho-chi-minh-kl8-vi.html"
    browser.get(url)
    time.sleep(3)
    html_source = browser.page_source  #lấy source của page bằng selenium
    soup = BeautifulSoup(html_source, 'html.parser')   #Tạo ra 1 beautyful soup object
    t = soup.find_all('a', {"class" : "job_link"})
    Name = []
    Salary = []
    Update_date = []
    Expired_date = []
    Experience = []
    Company = []
    if len(t) > 20:
        t = t[:20]
    for item in t:
        link = item.get('href')
        browser.get(link)
        time.sleep(3)
        html_source = browser.page_source  #lấy source của page bằng selenium
        soup = BeautifulSoup(html_source, 'html.parser')   #Tạo ra 1 beautyful soup object
        name = soup.find('h1', {"class" : "title"}).text
        company = soup.find('a', {"class" : "employer job-company-name"}).text
        Infor = soup.find_all('div', {"class" : "detail-box has-background"})
        for i in range(len(Infor)):
            Infor[i] = Infor[i].text
            Infor[i] = Infor[i].strip()
            Infor[i] = Infor[i].replace("\n","\t")
            Infor[i] = Infor[i].split("\t")
        for i in range(len(Infor)):
            for j in range(len(Infor[i])):
                Infor[i][j] = Infor[i][j].strip()
        new_info = []
        for i in range(len(Infor)):
            res = [ele for ele in Infor[i] if ele != '']
            new_info.append(res)
        Salary.append(new_sal[1][1])
        Update_date.append(new_sal[0][1])
        Expired_date.append(new_sal[1][-1])
        Experience.append(new_sal[1][3])
        Name.append(name)
        Company.append(company)
    Job_dataframe = pd.DataFrame({'Company' : Company, 'Name' : Name, 'Update_date' : Update_date,
                                 'Expired_date' : Expired_date, 'Experience' : Experience,'Salary' : Salary})
    tracks_dataframe.to_csv('Fresher_job.csv', index=False, header=True)  #Tạo file.csv mới với dataframe như trên
    return True

In [10]:
def Format_day(df, list_date):
    format_ = '%d/%m/%Y'
    n = len(df[list_date[0]])
    for item in list_date:
        for i in range(n):
            df[item][i] = datetime.datetime.strptime(track_df[item][i],format_)
    return df

In [11]:
def transform():
    in_csv_file_path = 'Fresher_job.csv'
    out_csv_file_path = 'Available_job.csv'

    df = pd.read_csv(in_csv_file_path)
    list_date = ['Expired_date', 'Update_date']

    df = Format_day(df,list_date)
    
    today = date.today()
    df = df[df.Expired_date <= today]
    
    df.to_csv(out_csv_file_path, index=False)
    return True

In [12]:
def email():
    port = 465  # For SSL
    import email, smtplib, ssl

    from email import encoders
    from email.mime.base import MIMEBase
    from email.mime.multipart import MIMEMultipart
    from email.mime.text import MIMEText

    subject = "Auto Mail from Python"
    sender_email = "tranbaotin2704@gmail.com"
    receiver_email = "tranbaotin2704@gmail.com"
    password = "**************"

    # Create a multipart message and set headers
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject
    # Add body to email

    filename = 'Available_job.csv'  # In same directory as script

    # Open PDF file in binary mode
    with open(filename, "rb") as attachment:
        # Add file as application/octet-stream
        # Email client can usually download this automatically as attachment
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())

    # Encode file in ASCII characters to send by email    
    encoders.encode_base64(part)

    # Add header as key/value pair to attachment part
    part.add_header(
        "Content-Disposition",
        f"attachment; filename= {filename}",
    )

    # Add attachment to message and convert message to string
    message.attach(part)
    text = message.as_string()

    # Log in to server using secure context and send email
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, text)

In [14]:
dag = DAG(
    'TinTran_dag',
    default_args={
        'email': ['tranbaotin2704@gmail.com'],
        'email_on_failure': True,
        'retries': 1,
        'retry_delay': timedelta(minutes=5),
    },
    description='First try on DAG',
    schedule_interval=timedelta(days=2),
    start_date=datetime(2023, 2, 8),
    tags=['tintran'])


extract_operator = PythonOperator(
    task_id='load_from_careerbuilder',
    python_callable=extract,
    dag=dag
)

transform_operator = PythonOperator(
    task_id='Filter_validate_row',
    python_callable=transform,
    dag=dag
)

email_operator = PythonOperator(
    task_id='email_to_me',
    python_callable=email,
    dag=dag
)

extract_operator >> transform_operator >> email_operator

<ipython-input-14-165e819067d9>:1 RemovedInAirflow3Warning: Param `schedule_interval` is deprecated and will be removed in a future release. Please use `schedule` instead.

<Task(PythonOperator): email_to_me>